# Takes constrained runs and dumps into a new file

This is what the people want!

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl

from dotenv import dotenv_values
from fair import __version__

In [ ]:
cal_v = dotenv_values("../../../.env")["CALIBRATION_VERSION"]
samples = int(dotenv_values("../../../.env")["PRIOR_SAMPLES"])
output_ensemble_size = int(dotenv_values("../../../.env")["POSTERIOR_SAMPLES"])
fair_v = dotenv_values("../../../.env")["FAIR_VERSION"]
constraint_set = dotenv_values("../../../.env")["CONSTRAINT_SET"]

assert fair_v == __version__

In [ ]:
df_cc=pd.read_csv(f'../../../output/fair-{fair_v}/v{cal_v}/priors/carbon_cycle.csv')
df_cr=pd.read_csv(f'../../../output/fair-{fair_v}/v{cal_v}/priors/climate_response_ebm3.csv')
df_aci=pd.read_csv(f'../../../output/fair-{fair_v}/v{cal_v}/priors/aerosol_cloud.csv')
df_ari=pd.read_csv(f'../../../output/fair-{fair_v}/v{cal_v}/priors/aerosol_radiation.csv')
df_ozone=pd.read_csv(f'../../../output/fair-{fair_v}/v{cal_v}/priors/ozone.csv')
df_scaling=pd.read_csv(f'../../../output/fair-{fair_v}/v{cal_v}/priors/forcing_scaling.csv')
df_1750co2=pd.read_csv(f'../../../output/fair-{fair_v}/v{cal_v}/priors/co2_concentration_1750.csv')

In [ ]:
valid_all = np.loadtxt(f'../../../output/fair-{fair_v}/v{cal_v}/posteriors/{constraint_set}/runids_rmse_reweighted_pass.csv').astype(np.int64)#[:1000]
valid_all

In [ ]:
seed = 1355763 + 399 * valid_all
seed

In [ ]:
params_out = pd.concat(
    (
        df_cr.loc[valid_all, :], 
        df_cc.loc[valid_all, :], 
        df_ari.loc[valid_all, :], 
        df_aci.loc[valid_all, :], 
        df_ozone.loc[valid_all, :], 
        df_scaling.loc[valid_all, :], 
        df_1750co2.loc[valid_all, :],
        pd.Series(seed, index=valid_all)
    )
, axis=1)

In [ ]:
pd.Series(seed, index=valid_all)

In [ ]:
params_out.columns = ['gamma', 'c1', 'c2', 'c3', 'kappa1', 'kappa2', 'kappa3', 'epsilon',
       'sigma_eta', 'sigma_xi', 'F_4xCO2', 'r0', 'rU', 'rT', 'rA', 'ari BC', 'ari CH4',
       'ari CO', 'ari N2O', 'ari NH3', 'ari NOx', 'ari OC', 'ari Sulfur', 'ari VOC',
       'ari Equivalent effective stratospheric chlorine', 'shape_so2',
       'shape_bc', 'shape_oc', 'beta', 'o3 CH4', 'o3 N2O',
       'o3 Equivalent effective stratospheric chlorine', 'o3 CO', 'o3 VOC', 'o3 NOx',
       'scale CO2', 'scale CH4', 'scale N2O', 'scale minorGHG', 'scale Stratospheric water vapour',
       'scale Contrails', 'scale Light absorbing particles on snow and ice', 'scale Land use',
       'scale Volcanic', 'solar_amplitude', 'solar_trend', 'co2_concentration_1750', 'seed']

In [ ]:
params_out.drop(columns=['ari CO'], inplace=True)

In [ ]:
params_out.to_csv(f'../../../output/fair-{fair_v}/v{cal_v}/posteriors/{constraint_set}/calibrated_constrained_parameters.csv')